# Code 1

This code acts to get item in selected brand

In [19]:
import json
import pandas as pd
import random
import time
import requests
import os

base_url = "https://catalog-api2.sociolla.com/search"
brand_slug = "558_cosrx"
limit = 50
skip = 0
keepLoop = True
data_list = []

filter_params = {
    "brand.slug": brand_slug
}

headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
}

while keepLoop:
    query_params = {
            "filter": json.dumps(filter_params),
            "limit": limit,
            "skip": skip
        }
    print
    response = requests.get(base_url, 
                            params=query_params, 
                            headers=headers)
        
    data = json.loads(response.text)
    products = data['data']
    
    # Function to get nested values
    def get_nested_value(dictionary, keys):
        for key in keys:
            dictionary = dictionary.get(key, {})
        return dictionary
    
    # Function to get price by combinations
    def get_price_by_combinations(product):
        combinations = product.get("combinations", [])
        price_by_combinations = {
            get_combination_name(entry.get("attributes", {})): entry.get("price")
            for entry in combinations
            if entry.get("status_item") == "active"
        }
        return "; ".join([f'"{key}": {value}' for key, value in price_by_combinations.items()])

    def get_combination_name(attributes):
        # Find the first key inside "attributes" that contains "name"
        name_key = next((key for key, value in attributes.items() if "name" in value), None)
        return attributes.get(name_key, {}).get("name") if name_key else None

    # Function to get categories as a semicolon-separated string
    def get_categories(product):
        categories = product.get("categories", [])
        category_names = [category.get("name") for category in categories]
        return "; ".join(category_names)

    # Loop through each product and populate the DataFrame
    if len(products)!=0:
        for product in products:
            data = {
                "product_name": product.get("name"),
                "product_id": product.get("id"),
                "beauty_point_earned": product.get("beauty_point_earned"),
                "price_range": product.get("price_range"),
                "price_by_combinations": get_price_by_combinations(product),
                "url": product.get("url_sociolla"),
                "active_date": product.get("active_for_sociolla_at"),
                "default_category": get_nested_value(product, ["default_category", "name"]),
                "categories": get_categories(product),
                "rating_types_str": ";".join(get_nested_value(product, ["default_category", "rating_types"])),
                "average_rating": get_nested_value(product, ["review_stats", "average_rating"]),
                "total_reviews": get_nested_value(product, ["review_stats", "total_reviews"]),
                "average_rating_by_types": "; ".join([f'"{key}": {value}' for key, value in get_nested_value(product, ["review_stats", "average_rating_by_types"]).items()]),
                "total_recommended_count": get_nested_value(product, ["review_stats", "total_recommended_count"]),
                "total_repurchase_maybe_count": get_nested_value(product, ["review_stats", "total_repurchase_maybe_count"]),
                "total_repurchase_no_count": get_nested_value(product, ["review_stats", "total_repurchase_no_count"]),
                "total_repurchase_yes_count": get_nested_value(product, ["review_stats", "total_repurchase_yes_count"]),
                "total_in_wishlist": product.get("total_wishlist")
            }
            data_list.append(data)
    else:
        # Specify the result folder name
        result_folder_name = "result_6_Dec"

        # Create the result folder path
        result_folder = os.path.join("..", "result", result_folder_name)

        # Ensure that the result folder exists, if not, create it
        os.makedirs(result_folder, exist_ok=True)

        # Ensure that the result folder exists, if not, create it
        if not os.path.exists(result_folder):
            os.makedirs(result_folder)

        # Create file name
        file_name = f"products_{brand_slug}.csv"
        
        # Save the aggregated dataframe to CSV
        result_file_path = os.path.join(result_folder, file_name)
        df_products = pd.DataFrame(data_list)
        df_products.to_csv(result_file_path, index=False)
        
        # Stop the loop
        keepLoop = False
        print("selesai")
        break

    rand_numb = abs(random.gauss(3, 1))
    time.sleep(rand_numb)
    print("\t action delayed for ", rand_numb, "second")
    print("\t scraped [", len(data_list), "] product(s)")
    print("")
    skip += limit

	 action delayed for  1.1926214098876515 second
	 scraped [ 50 ] product(s)

	 action delayed for  4.547150673920211 second
	 scraped [ 95 ] product(s)

selesai


# Code 2

This code acts to get the entire brand products

In [1]:
import json
import os
import pandas as pd
import random
import time
import requests

brands = pd.read_csv('/Users/hafizzzh/Documents/AAA Project Skripsi/ecommerce_SA/result/brand_sociolla.csv')

def scrape_brand_data(brand_slug, limit=50):
    base_url = "https://catalog-api2.sociolla.com/search"
    skip = 0
    data_list = []

    filter_params = {
        "brand.slug": brand_slug
    }

    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'
    }

    while True:
        query_params = {
            "filter": json.dumps(filter_params),
            "limit": limit,
            "skip": skip
        }
        print
        response = requests.get(base_url, 
                                params=query_params, 
                                headers=headers)
            
        data = json.loads(response.text)
        products = data['data']
        
        # Function to get nested values
        def get_nested_value(dictionary, keys):
            for key in keys:
                dictionary = dictionary.get(key, {})
            return dictionary
        
        # Function to get price by combinations
        def get_price_by_combinations(product):
            combinations = product.get("combinations", [])
            price_by_combinations = {
                get_combination_name(entry.get("attributes", {})): entry.get("price")
                for entry in combinations
                if entry.get("status_item") == "active"
            }
            return "; ".join([f'"{key}": {value}' for key, value in price_by_combinations.items()])

        def get_combination_name(attributes):
            # Find the first key inside "attributes" that contains "name"
            name_key = next((key for key, value in attributes.items() if "name" in value), None)
            return attributes.get(name_key, {}).get("name") if name_key else None

        # Function to get categories as a semicolon-separated string
        def get_categories(product):
            categories = product.get("categories", [])
            category_names = [category.get("name") for category in categories]
            return "; ".join(category_names)

        # Loop through each product and populate the DataFrame
        if len(products)!=0:
            for product in products:
                data = {
                    "brand_name": brand_slug,
                    "product_name": product.get("name"),
                    "product_id": product.get("id"),
                    "beauty_point_earned": product.get("beauty_point_earned"),
                    "price_range": product.get("price_range"),
                    "price_by_combinations": get_price_by_combinations(product),
                    "url": product.get("url_sociolla"),
                    "active_date": product.get("active_for_sociolla_at"),
                    "default_category": get_nested_value(product, ["default_category", "name"]),
                    "categories": get_categories(product),
                    "rating_types_str": ";".join(get_nested_value(product, ["default_category", "rating_types"])),
                    "average_rating": get_nested_value(product, ["review_stats", "average_rating"]),
                    "total_reviews": get_nested_value(product, ["review_stats", "total_reviews"]),
                    "average_rating_by_types": "; ".join([f'"{key}": {value}' for key, value in get_nested_value(product, ["review_stats", "average_rating_by_types"]).items()]),
                    "total_recommended_count": get_nested_value(product, ["review_stats", "total_recommended_count"]),
                    "total_repurchase_maybe_count": get_nested_value(product, ["review_stats", "total_repurchase_maybe_count"]),
                    "total_repurchase_no_count": get_nested_value(product, ["review_stats", "total_repurchase_no_count"]),
                    "total_repurchase_yes_count": get_nested_value(product, ["review_stats", "total_repurchase_yes_count"]),
                    "total_in_wishlist": product.get("total_wishlist")
                }
                data_list.append(data)
        else:
            print(f"Scraped {len(data_list)} products for brand {brand_slug}")
            break

        rand_numb = abs(random.gauss(3, 1))
        time.sleep(rand_numb)
        skip += limit

    return pd.DataFrame(data_list)

# Specify the result folder name
result_folder_name = "result_8_Dec"

# Create the result folder path
result_folder = os.path.join("..", "result", result_folder_name)

# Ensure that the result folder exists, if not, create it
os.makedirs(result_folder, exist_ok=True)

# Ensure that the result folder exists, if not, create it
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

# Create an empty dataframe to store all products
meta_products = pd.DataFrame()

# Assuming 'brands' is your DataFrame
for brand_slug in brands['Slug']:
    print(f"Fetching products for brand: {brand_slug}")
    brand_products = scrape_brand_data(brand_slug)
    meta_products = pd.concat([meta_products, brand_products])
    
    # Save the current product data
    result_file_path = os.path.join(result_folder, f"products_{brand_slug}.csv")
    brand_products.to_csv(result_file_path, index=False)
    print("")

# Save the aggregated dataframe to CSV
result_file_path = os.path.join(result_folder, f"products_all_brands.csv")
meta_products.to_csv(result_file_path, index=False)

print(f"\nAll products saved to: {result_file_path}")


Fetching products for brand: 796_3ce
Scraped 26 products for brand 796_3ce

Fetching products for brand: 751_abib
Scraped 23 products for brand 751_abib

Fetching products for brand: 730_acwell
Scraped 20 products for brand 730_acwell

Fetching products for brand: 15654_adara-cosmetics
Scraped 18 products for brand 15654_adara-cosmetics

Fetching products for brand: 584_aeris-beaute
Scraped 59 products for brand 584_aeris-beaute

Fetching products for brand: 15703_alatte
Scraped 3 products for brand 15703_alatte

Fetching products for brand: 15542_alchemist-fragrance
Scraped 10 products for brand 15542_alchemist-fragrance

Fetching products for brand: 1350_allglows
Scraped 3 products for brand 1350_allglows

Fetching products for brand: 15700_amuse
Scraped 10 products for brand 15700_amuse

Fetching products for brand: 2019_anessa
Scraped 41 products for brand 2019_anessa

Fetching products for brand: 15689_anua
Scraped 9 products for brand 15689_anua

Fetching products for brand: 814_

In [2]:
print(meta_products.columns)

Index(['brand_name', 'product_name', 'product_id', 'beauty_point_earned',
       'price_range', 'price_by_combinations', 'url', 'active_date',
       'default_category', 'categories', 'rating_types_str', 'average_rating',
       'total_reviews', 'average_rating_by_types', 'total_recommended_count',
       'total_repurchase_maybe_count', 'total_repurchase_no_count',
       'total_repurchase_yes_count', 'total_in_wishlist'],
      dtype='object')


In [3]:
print(meta_products)

         brand_name                  product_name  product_id   
0           796_3ce       MULTI EYE COLOR PALETTE       97802  \
1           796_3ce               VELVET LIP TINT       97810   
2           796_3ce                     LIP COLOR       97822   
3           796_3ce  MINI MULTI EYE COLOR PALETTE       97833   
4           796_3ce                    FACE BLUSH       97801   
..              ...                           ...         ...   
4   273_yves-rocher    Ambre Noir Eau De Toilette        2158   
5   273_yves-rocher  Soin Stimulating Conditioner       64174   
6   273_yves-rocher              Repair Hair Mask       36357   
7   273_yves-rocher   Hand Cream Olive Petitgrain       36404   
8   273_yves-rocher                    Color Mask       64153   

    beauty_point_earned              price_range price_by_combinations   
0                   130  Rp 555.000 - Rp 687.000                        \
1                    50               Rp 264.000                       